In [1]:
# Imports for making dataframe
import pandas as pd
import numpy as np
from time import sleep

In [2]:
# Import for crawling Goodreads
import requests

In [3]:
# Imports for XML
from xml.etree import ElementTree

In [4]:
# Import for finding texts in HTML
from bs4 import BeautifulSoup

In [5]:
# Imports for seeing full text reviews
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.action_chains import ActionChains

In [6]:
# Goodreads API Developer key
key = 'e8m0Y7q3q9psBCGuwTDRUA'
secret = 'sQzxhCY5eEGBUJLUwOrKnmPOGUIRUvSemqntp6ZSDJU'

In [7]:
## NOTE: FILL IN BEFORE RUNNING CODE
# Goodreads login
email = ''
password = ''

In [8]:
## NOTE: FILL IN BEFORE RUNNING CODE
# Load Amazon data to find reviews of same books from Goodreads
amazon = pd.read_csv('asinOnly6.csv')
amazon

,asin
0,0062089099
1,0062089196
2,0062089234
3,0062089285
4,0062089374
...,...
2967,0062391739
2968,0062391755
2969,0062391763
2970,0062391771


In [9]:
# Unique set of books to crawl from Goodreads
books = set(amazon.asin)
print(len(books))

2972


### NOTE: COMMENT OUT FOR FULL SET
## TEST SUBSET
test = set()
for i,val in enumerate(books):
    if i == 3:
        break
    test.add(val)
books = test
books

In [10]:
# Create empty dataframe to store Goodreads reviews & ratings
column_names = ['overall', 'reviewTime', 'asin', 'reviewText']
df = pd.DataFrame(columns = column_names)
print(df)

Empty DataFrame
Columns: [overall, reviewTime, asin, reviewText]
Index: []


In [11]:
# Log in Goodreads
driver = webdriver.Chrome(r"C:\\Users\\Tien Nguyen\\Stats_170AB\\chromedriver.exe")
driver.get ('https://www.goodreads.com/user/sign_in?source=home')
driver.find_element_by_name('user[email]').send_keys(email)
driver.find_element_by_name('user[password]').send_keys(password)
driver.find_element_by_name('next').click()                            

In [12]:
import os
REVIEW_FOLDER = "C:\\Users\\Tien Nguyen\\Stats_170AB\\goodreads_review_dataset\\"

In [13]:
# Get most popular reviews for each book (10 if there are more than 10 reviews, or all reviews if less)
for book in books:
    # if the reviews for the book is downloaded, skip it
    review_path = os.path.join(REVIEW_FOLDER, book + ".txt")
    if os.path.exists(review_path):
        continue
    
    f = open(review_path, 'w')
        
    # Try to look up the book in Goodreads. If a book isn't found it will not be added to the dataset -> DATA CLEANING LATER
    try:
        #print('TRYING BOOK:', book)
        
        # Search up book on Goodreads
        res = requests.get('https://www.goodreads.com/book/isbn', params={'format': 'xml', 'key': key, 'isbn': book})
        
        # Get XML from API
        root = ElementTree.fromstring(res.text)    
        
        # Find iframe from XML
        iframe = root.find('book').find('reviews_widget').text
        
        # Parse HTML of XML to find link to reviews
        soup = BeautifulSoup(iframe, 'html.parser')
        url = soup.find('iframe')['src']
        
        # Open link to reviews using Selenium
        driver.get(str(url))

        # PAGE 1: Expand each review to save full text & metadata
        # If a book doesn't have reviews, it will not be added to the dataset -> DATA CLEANING LATER
        links = driver.find_elements_by_link_text('...more') 

        for link in links:
            # Open review in new tab
            ActionChains(driver).move_to_element(link).perform()
            link.click()
            
            # Switch to new tab
            driver.switch_to.window(driver.window_handles[1])
            s = BeautifulSoup(driver.page_source, 'html.parser')
            
            # Use BeautifulSoup to check & save reviewTime
            time = s.find('span', itemprop='datePublished')
            
            # Use BeautifulSoup to save rating
            rating = s.find('meta', itemprop='ratingValue')
            
            # Use BeautifulSoup to save text review
            review = s.find('div', itemprop='reviewBody')
            
            # Add review to dataframe
            #df = df.append({'overall':rating.get('content'), 'reviewTime':time.text.strip(), 'asin':book, 
            #                'reviewText':review.text.strip()}, ignore_index=True)
            
            print("{}\n{}\n{}\n{}\{}n".format(rating.get('content'),
                                                time.text.strip(),
                                                book,
                                                review.text.strip()))
            f.write("{}\n{}\n{}\n{}\n".format(rating.get('content'),
                                                time.text.strip(),
                                                book,
                                                review.text.strip()))
                
            driver.close()
            driver.switch_to.window(driver.window_handles[0])
            
        # PAGE 2: Expand each review to save full text & metadata
        # If a book doesn't have reviews, it will not be added to the dataset -> DATA CLEANING LATER
        driver.find_element_by_link_text('2').click()
        links = driver.find_elements_by_link_text('...more') 

        for link in links:
            # Open review in new tab
            ActionChains(driver).move_to_element(link).perform()
            link.click()
            
            # Switch to new tab
            driver.switch_to.window(driver.window_handles[1])
            s = BeautifulSoup(driver.page_source, 'html.parser')
            
            # Use BeautifulSoup to check & save reviewTime
            time = s.find('span', itemprop='datePublished')
            
            # Use BeautifulSoup to save rating
            rating = s.find('meta', itemprop='ratingValue')
            
            # Use BeautifulSoup to save text review
            review = s.find('div', itemprop='reviewBody')
            
            # Add review to dataframe
            #df = df.append({'overall':rating.get('content'), 'reviewTime':time.text.strip(), 'asin':book, 
            #                'reviewText':review.text.strip()}, ignore_index=True)
            f.write("{}\n{}\n{}\n{}\n".format(rating.get('content'),
                                                time.text.strip(),
                                                book,
                                                review.text.strip()))
            
                
            driver.close()           
            driver.switch_to.window(driver.window_handles[0])
            
        # PAGE 3: Expand each review to save full text & metadata
        # If a book doesn't have reviews, it will not be added to the dataset -> DATA CLEANING LATER
        driver.find_element_by_link_text('3').click()
        links = driver.find_elements_by_link_text('...more') 

        for link in links:
            # Open review in new tab
            ActionChains(driver).move_to_element(link).perform()
            link.click()
            
            # Switch to new tab
            driver.switch_to.window(driver.window_handles[1])
            s = BeautifulSoup(driver.page_source, 'html.parser')
            
            # Use BeautifulSoup to check & save reviewTime
            time = s.find('span', itemprop='datePublished')
            
            # Use BeautifulSoup to save rating
            rating = s.find('meta', itemprop='ratingValue')
            
            # Use BeautifulSoup to save text review
            review = s.find('div', itemprop='reviewBody')
            
            # Add review to dataframe
            #df = df.append({'overall':rating.get('content'), 'reviewTime':time.text.strip(), 'asin':book, 
            #                'reviewText':review.text.strip()}, ignore_index=True)
            
            # Save a review to a file
            f.write("{}\n{}\n{}\n{}\n".format(rating.get('content'),
                                                time.text.strip(),
                                                book,
                                                review.text.strip()))
            
            driver.close()           
            driver.switch_to.window(driver.window_handles[0])
            
        f.close()
    except:
        f.close()
        pass
    
    sleep(2)
        #print('   NOT FOUND')

KeyboardInterrupt: 

In [ ]:
df

In [ ]:
## NOTE: CHANGE FILE NAME BEFORE RUNNING CODE  
df.to_csv('grData6.csv',index=False)